J1:
Swedish: ['Morän', 'Postglacial sand--grus', 'Svallsediment, grus--block', 'Torv', 'Isälvssediment', 'Lera--silt']
Finnish: ['Moreeni', 'Jääkauden jälkeinen hiekka--sora', 'Huuhdottu sedimentti, sora--kivet', 'Turve', 'Sulamisvesisedimentti', 'Savi--lieju']
English: ['Moraine', 'Postglacial sand--gravel', 'Washed sediment, gravel--boulders', 'Peat', 'Glaciofluvial sediment', 'Clay--silt']

J2:
Swedish: ['Fyllning', 'Isälvssediment', 'Torv', 'Postglacial sand', 'Älvsediment, sand', 'Vatten', 'Morän', 'Lera--silt', 'Berg']
Finnish: ['Täyttömaa', 'Sulamisvesisedimentti', 'Turve', 'Jääkauden jälkeinen hiekka', 'Jokisedimentti, hiekka', 'Vesi', 'Moreeni', 'Savi--lieju', 'Kallio']
English: ['Fill', 'Glaciofluvial sediment', 'Peat', 'Postglacial sand', 'Fluvial sediment, sand', 'Water', 'Moraine', 'Clay--silt', 'Bedrock']

J1_J2:
Swedish: [

In [2]:
import tools
import rasterio
import geopandas as gpd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from catchment import stem_volume_to_LAI
from rasterio.fill import fillnodata
from tools import burn_water_dem, read_AsciiGrid, write_AsciiGrid
from rasterio.plot import show
import rasterio
from scipy.stats import mstats
import rasterio.windows
import sys


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
fd = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP/2010/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/'
p = os.path.join(fd, '*.tif')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.asc'
    tools.open_raster_with_subset(file, out_fp, subset, plot=False, save_in='asc')

In [34]:
fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/'
p = os.path.join(fd, '*.asc')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.asc'
    with rasterio.open(file) as src:
        data = src.read(1)
        meta = src.meta.copy()
        nodata_1 = meta['nodata']
        nodata_2 = -9999
        data[data == nodata_1] = nodata_2
        data[data == nodata_2] = 0.0
        meta.update({"nodata":nodata_2})
    with rasterio.open(out_fp, 'w', **meta) as dst:
        src = dst.write(data, 1)

### INTERPOLATING DEM HOLES

In [13]:
# INTERPOLATING HOLES
fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM.tif'
with rasterio.open(fp) as src:
    profile = src.profile
    arr = src.read(1)
    arr_filled = fillnodata(arr, mask=src.read_masks(1), max_search_distance=5, smoothing_iterations=0)

out_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_filled.tif'
with rasterio.open(out_fp, 'w', **profile) as dest:
    dest.write_band(1, arr_filled)

In [43]:
dem_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_filled.tif'
stream_mask_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/derivatives_0case/streams_d8_5ha.asc'
stream_ele_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/derivatives_1case/streams_d8_5ha_elev.asc'

# Step 1: Read DEM and stream mask
with rasterio.open(dem_path) as dem_src, rasterio.open(stream_mask_path) as stream_src:
    dem = dem_src.read(1)  # DEM data (0.5m resolution)
    stream_mask = stream_src.read(1)  # Stream mask data (25m resolution)
    
    # Get metadata of the stream mask raster
    stream_meta = stream_src.meta.copy()
    stream_meta.update(crs=dem_src.crs)
    
# Step 2: For each stream cell in stream_mask, find corresponding DEM cells
# Stream mask has lower resolution, so we calculate corresponding DEM grid-cells
dem_transform = dem_src.transform
stream_transform = stream_src.transform

stream_elev = np.full_like(stream_mask, np.nan, dtype=np.float32)

stream_cells = np.where(stream_mask == 1)  # Assuming 1 marks stream locations


# Iterate over each stream cell in the lower resolution stream mask
for i, j in zip(*stream_cells):
    # Create a 1x1 window for the current stream cell in the stream mask
    stream_window = rasterio.windows.Window(col_off=j, row_off=i, width=1, height=1)
    
    # Get the bounds of the stream cell in the stream mask's coordinate system
    stream_bounds = rasterio.windows.bounds(stream_window, stream_transform)
    
    # Convert stream bounds to corresponding DEM window using index notation
    dem_col_start, dem_row_start = ~dem_transform * (stream_bounds[0], stream_bounds[3])  # (left, top)
    dem_col_end, dem_row_end = ~dem_transform * (stream_bounds[2], stream_bounds[1])  # (right, bottom)

    #if j == 114:
    #    print(dem_transform)
    #    print('')
    #    print(~dem_transform)
    #    print('')
    #    print('row', dem_row_start, dem_row_end)
    #    print('')
    #    print('col', dem_col_start, dem_col_end)
    #    sys.exit()
        
    # Convert to integer indices for the DEM grid
    dem_row_start, dem_row_end = int(np.floor(dem_row_start)), int(np.ceil(dem_row_end))
    dem_col_start, dem_col_end = int(np.floor(dem_col_start)), int(np.ceil(dem_col_end))
    
    # Step 3: Extract DEM values within the corresponding DEM cells
    dem_subset = dem[dem_row_start:dem_row_end, dem_col_start:dem_col_end]
            
    del dem_row_start, dem_row_end, dem_col_start, dem_col_end

        
    # Flatten and remove np.nan values from DEM subset
    valid_dem_values = dem_subset[np.isfinite(dem_subset)]

    
    if len(valid_dem_values) > 0:
        # Step 4: Calculate the first quantile (0.1)
        #quantile_value = mstats.mquantiles(valid_dem_values, prob=0.1)[0]
        quantile_value = np.quantile(valid_dem_values, 0.02)
        # Assign the quantile value to the stream depth for the current stream cell
        stream_elev[i, j] = quantile_value
    
# Step 5: Save the new raster with the same metadata as the stream mask
stream_meta.update(dtype=rasterio.float32, count=1)

with rasterio.open(stream_ele_path, 'w', **stream_meta) as dst:
    dst.write(stream_elev, 1)

In [44]:
stream_ele_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/derivatives_1case/streams_d8_5ha_elev.asc'
dem_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/derivatives_0case/orig_dem.asc'
stream_depth_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/derivatives_1case/streams_d8_5ha_depths.asc'

# Step 1: Open the stream elevation and DEM files
with rasterio.open(stream_ele_path) as stream_src:
    stream_ele = stream_src.read(1)  # Read stream elevation
    stream_meta = stream_src.meta.copy()  # Get metadata from stream elevation

with rasterio.open(dem_path) as dem_src:
    dem_data = dem_src.read(1)  # Read DEM data

# Step 2: Create a mask for valid DEM values
dem_mask = np.isfinite(dem_data)

# Step 3: Calculate stream depth only where DEM is valid
# Only calculate where stream elevation is valid (not NaN) and DEM is valid
stream_depth = np.where(dem_mask & np.isfinite(stream_ele), stream_ele - dem_data, np.nan)

stream_depth[stream_depth > 0] = 0
stream_depth[stream_depth < -15 ] = -15

# Step 4: Save the stream depth as a new raster
stream_meta.update(dtype=rasterio.float32, count=1)

with rasterio.open(stream_depth_path, 'w', **stream_meta) as dst:
    dst.write(stream_depth, 1)